In [ ]:
# 임포트
import pandas as pd
import numpy as np
import langchain 
import requests
from bs4 import BeautifulSoup

In [ ]:
# config
# 수집데이터
keywords = {}

searchKeywordFormat = {
        'daumOpenKeywords' : {},
        'daumSuggestKeywords' : {},
        'naverOpenKeywords' : {},
        'googleSuggestKeywords' : {},
        'mergedKeywords' : {},
        }    

keywordValuationFormat = {
    'searchVolume' : '',
    'countDaumADs' : '',
    'tistoryRank' : '',    
    'provider' : ''  
}


In [ ]:
# 검색어
searchWord = '와인얼룩'
keywords[searchWord] = searchKeywordFormat.copy()

# 주요 크롤링 대상
def createDaumURL(word) :
    return f'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={word}'
def createNaverURL(word) :
    return f'https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={word}'
def createGoogleURL(word) :
    return f'https://www.google.com/search?q={word}&sca_esv=a19c4e940f5aec6f&rlz=1C1IBEF_koKR958KR958&ei=UjTbZePvLeGP2roPzY2qiAo&udm=&ved=0ahUKEwjj4I7ZwMaEAxXhh1YBHc2GCqEQ4dUDCBA&uact=5&oq={searchWord}&gs_lp=Egxnd3Mtd2l6LXNlcnAiD-yKpO2LsOu4jOyeoeyKpDIFEC4YgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAuGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEC4YgAQyFBAuGIAEGJcFGNwEGN4EGN8E2AEBSLoOUIQFWNQLcAN4ApABA5gBjAGgAdkLqgEEMC4xMrgBA8gBAPgBAZgCCKACvgTCAgQQABhHwgIEEAAYA8ICCxAAGIAEGLEDGIMBwgIEEC4YA8ICERAuGIAEGLEDGIMBGMcBGNEDwgIOEC4YgAQYsQMYxwEY0QPCAgoQLhiABBiKBRhDwgIKEAAYgAQYigUYQ8ICGRAuGIAEGIoFGEMYlwUY3AQY3gQY3wTYAQHCAgsQLhiABBixAxiDAcICChAuGEMYgAQYigXCAhkQLhhDGIAEGIoFGJcFGNwEGN4EGN8E2AEBmAMAiAYBkAYKugYGCAEQARgUkgcDNC40&sclient=gws-wiz-serp'

daumURL = createDaumURL(searchWord)
naverURL = createNaverURL(searchWord)
googleURL = createGoogleURL(searchWord)
URLS = [daumURL, naverURL, googleURL]

In [ ]:
# 오픈키워드 긁어오기

## 함수정의
def getOpenKeywords(url, tag) :
    # 웹페이지 가져오기
    response = requests.get(url)
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')
  
    words = soup.select(tag)
    wordsLength = len(words)
    if debug : print('length', wordsLength)
    
    if(wordsLength > 0) :
        result = [word.text.replace(' ','') for word in words]
        return result

def keywordFormatting(words, format, provider) :
    result = {}
    for word in words :
        result[word] = format.copy()
        result[word]['provider'] = provider
    return result

## 태그정의
daumOpenKeywordsTag = '#netizen_lists_top span' # 관련키워드
naverOpenKeywordsTag = 'a.title_link'

## 불러오기
daumOpenKeywords = getOpenKeywords(daumURL, daumOpenKeywordsTag)
naverOpenKeywords = getOpenKeywords(naverURL, naverOpenKeywordsTag)

# db저장
limit = 5 # 오픈키워드는 그다지 의미가 없어서 저장하는 갯수 제한
if(len(daumOpenKeywords) > 0):
    keywords[searchWord]['daumOpenKeywords'] = keywordFormatting(daumOpenKeywords[:limit], keywordValuationFormat, 'daumOpenKeywords')
if(len(naverOpenKeywords) > 0): 
    keywords[searchWord]['naverOpenKeywords'] =keywordFormatting( naverOpenKeywords[:limit], keywordValuationFormat, 'naverOpenKeywords')

In [ ]:

# 서제스트 키워드 긁어오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [ ]:
# 서제스트 키워드 긁어오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## config
daumClickElementCSSQuery = '#q.tf_keyword'
googleClickElementCSSQuery = '#searchform textarea.gLFyf'

daumDataCSSQuery = "span.txt_query"
googleDataCSSQuery = 'div.wM6W7d span b'



## 로직정의 및 실행
def seleniumClickAndCrawling(driver, clickElementCSSQuery, dataCSSQuery) :
    # 2. 검색창 클릭
    search_box = driver.find_element(By.CSS_SELECTOR, clickElementCSSQuery)
    search_box.click()

    # 잠시 대기 (동적 콘텐츠 로딩을 위함)
    time.sleep(1)

    # 3. "아메리카노" 텍스트가 포함된 태그들 선택 및 텍스트 수집
    elements = driver.find_elements(By.CSS_SELECTOR, dataCSSQuery)
    result = [element.text.replace(' ', '') for element in elements]

    return result




# 셀레니움 실행
driver = webdriver.Chrome()

try:
    # 1. 대상 웹사이트 접속
    driver.get(createDaumURL(searchWord))
    daumSuggestKeywords = seleniumClickAndCrawling(driver, daumClickElementCSSQuery, daumDataCSSQuery)
    driver.get(createGoogleURL(searchWord))
    googleSuggestKeywords = seleniumClickAndCrawling(driver, googleClickElementCSSQuery, googleDataCSSQuery)
    # 에러 핸들링
except Exception as e:
    if debug : print('error occured', e)
# 셀레니움 종료
finally:
    driver.quit()
    

#데이터 통합
keywords[searchWord]['daumSuggestKeywords'] = keywordFormatting(daumSuggestKeywords, keywordValuationFormat, 'daumSuggestKeywords')
keywords[searchWord]['googleSuggestKeywords'] = keywordFormatting(googleSuggestKeywords, keywordValuationFormat, 'googleSuggestKeywords')

In [ ]:
for key, value in keywords[searchWord].items() :
    if debug : print(key,value)

In [ ]:
# 데이터 정리하기
# 하나로 뭉치기

a = keywords[searchWord]['daumOpenKeywords']
b = keywords[searchWord]['daumSuggestKeywords']
c = keywords[searchWord]['googleSuggestKeywords']
d = keywords[searchWord]['naverOpenKeywords']

merged_dict = {**a, **b, **c, **d} # 중복제거
keywords[searchWord]['mergedKeywords'] = merged_dict

In [ ]:
pd.DataFrame(keywords[searchWord]['mergedKeywords'])

In [ ]:
#광고관리하기

#config_광고나타내는 태그 확인하기
DAUM_AD_normal = '.list_ad' #와이드광고, 프리미엄광고
DAUM_AD_special = '#splinkColl .list_info.mg_cont.clear' # 스페셜광고
DAUM_REL_KEYWORDS = '#netizen_lists_top span' # 관련키워드

# 로직정의하기
def countDaumADs(url, word, tags) :
    response = requests.get(url) # 페이지 가져오기
    soup = BeautifulSoup(response.text, 'html.parser') # BeautifulSoup 객체 생성

    num_advertisement = 0
    for tag in tags :        
        contents = soup.select(tag)     # 광고섹션
        contentsLength = len(contents)     # 광고섹션갯수
        for i in range(contentsLength) :
            num_advertisement += len(contents[i].find_all(recursive=False))     # 광고갯수합산
    #디버그용 if debug : print(word,'광고갯수 : ',num_advertisement)
    return num_advertisement

for key, val in keywords[searchWord]['mergedKeywords'].items() :
    count = countDaumADs(createDaumURL(key), key, [DAUM_AD_special,DAUM_AD_normal])
    keywords[searchWord]['mergedKeywords'][key]['countDaumADs'] = count

In [ ]:
# 구글 서치볼륨 확인하기


# 로직정의
def countGoogleSearchVolume(driver, CSSQuery,) :
# 웹 드라이버 초기화 (Chrome 사용을 가정)
        elements = driver.find_elements(By.CSS_SELECTOR, CSSQuery)
        result = [element.text for element in elements]
        if debug : print('results', result)
        # 필요한 글자처리
        result = result[0]
        result = result.split(' ')[2]
        result = int(result.replace('개', '').replace(',',''))
        return result

    
    

# tag정의
tag = 'div#result-stats'


# 로직실행 및 저장
driver = webdriver.Chrome()
try:
    for key, val in keywords[searchWord]['mergedKeywords'].items() :
        
            # 1. 대상 웹사이트 접속
            driver.get(createGoogleURL(key))
            volume = countGoogleSearchVolume(driver, tag)
            if debug : print(volume)
            keywords[searchWord]['mergedKeywords'][key]['searchVolume'] = volume
except Exception as e:
    if debug : print('error occured', e)
# 셀레니움 종료
finally:
    driver.quit()

    
    
    

In [ ]:
# 구글 서치볼륨 확인하기

# 로직정의
def countGoogleSearchVolume(driver, CSSQuery) :
        # 서치볼륨태그 확인하기
        result = []
        elements = driver.find_elements(By.CSS_SELECTOR, CSSQuery)
        result = [element.text for element in elements]        
        # 필요한 글자처리
        result = result[0]
        result = result.split(' ')[2]
        result = int(result.replace('개', '').replace(',',''))
        # 로직 종료
        return result


In [ ]:
# 단일실행 (countGoogleSearchVolume 실행확인용)

# config
# tag = 'div#result-stats'
# url = createGoogleURL('박지성')
# driver = webdriver.Chrome()


# try:
#     driver.get(url)
#     volume = countGoogleSearchVolume(driver, tag)    
# except Exception as e:
#     if debug : print('error occured', e)
# finally:
#     driver.quit()
    
# if debug : print(volume)

In [ ]:
# 구글 검색 섹션 순서 확인하기.
# 로직정의
def checkGoogleSectionOrder(driver, CSSQuery ) :                
    elements = driver.find_elements(By.CSS_SELECTOR, CSSQuery)        
    result = [element.get_attribute('href') for element in elements]
    rank = 1
    try :
        for href in result :
            if 'tistory' in href :            
                return rank
            else :
                rank += 1
        return rank
    except Exception as e :
        if debug : print('in Ranking TISTORY PRIMARY, error occuered',e)
        return "not Defined"


In [ ]:
# 단일실행 (checkGoogleSectionOrder 실행확인용)


# tag = 'div#rso.dURPMd > div.MjjYud a'

# url = createGoogleURL(searchWord)
# driver = webdriver.Chrome()
# try:
#     # 1. 대상 웹사이트 접속
#     driver.get(url)
#     rank = checkGoogleSectionOrder(driver,tag)
# except Exception as e:
#     if debug : print('error occured', e)
# finally:
#     driver.quit()


In [ ]:
# config(countGoogleSearchVolume)
volumeCSSQuery = 'div#result-stats'

# config(checkGoogleSectionOrder)
rankCSSQuery = 'div#rso.dURPMd > div.MjjYud a'

# 셀레니움 실행
driver = webdriver.Chrome()

for key, val in keywords[searchWord]['mergedKeywords'].items() :
    driver.get(createGoogleURL(key)) # 디렉션
    
    volume = countGoogleSearchVolume(driver,volumeCSSQuery)    # 검색량 측정
    rank = checkGoogleSectionOrder(driver, rankCSSQuery) # 랭크 측정
    
    keywords[searchWord]['mergedKeywords'][key]['searchVolume'] = volume # 검색량 저장
    keywords[searchWord]['mergedKeywords'][key]['tistoryRank'] = rank # 랭크 저장

In [ ]:
df = pd.DataFrame(keywords[searchWord]['mergedKeywords'])
display(df)
# df.to_csv('test.csv', encoding='cp949')


완성하기(반복작업하기)


In [2]:
# 임포트
# 임포트
import pandas as pd
import numpy as np
import langchain 
import requests
from bs4 import BeautifulSoup

# 서제스트 키워드 긁어오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [10]:
# 임포트 2
from create_URL import createDaumURL
from create_URL import createGoogleURL
from create_URL import createYoutubeURL
from create_URL import createNaverURL

from getOpenKeywords import getOpenKeywords
from keywordFormatting import keywordFormatting
from seleniumClickAndCrawling import seleniumClickAndCrawling
from countDaumADs import countDaumADs
from count_GoogleSearchVolume import countGoogleSearchVolume
from checkGoogleSectionOrder import checkGoogleSectionOrder
from getGoogleOpenKeywords import getGoogleOpenKeywords
from limitLength import limitLength

In [23]:
# config _ format 설정
# config
# 수집데이터
keywords = {}

providers = ['daumOpenKeywords',
'daumSuggestKeywords',
'googleSuggestKeywords',
'googleOpenKeywords',
# 'youtubeSuggestKeywords',
]
benchmarks = ['searchVolume','countDaumADs','tistoryRank','provider']

searchKeywordFormat = {}

# searchKeywordFormat = {
        # 'daumOpenKeywords' : {},
        # 'daumSuggestKeywords' : {},
        # 'naverOpenKeywords' : {},
        # 'googleSuggestKeywords' : {},
        # 'googleOpenKeywords':{},
        # 'youtubeSuggestKeywords':{},
        # 'mergedKeywords' : {},
        # }    

keywordValuationFormat = {
    'searchVolume' : '',
    'countDaumADs' : '',
    'tistoryRank' : '',    
    'provider' : '',
    'depth' : ''  
}


In [24]:
# config _ CSSSelector설정

## 오픈키워드 선택자
# daumOpenKeywordsTag = ['#netizen_lists_top span', '#recomm_lists_top a'] # 관련키워드
daumOpenKeywordsTag =  'a.keyword'# 관련키워드

naverOpenKeywordsTag = 'a.title_link'

# 서제스트키워드 선택자
daumClickElementCSSQuery = '#q.tf_keyword'
googleClickElementCSSQuery = '#searchform textarea.gLFyf'
youtubeClickElementCSSQuery = 'input#search'

daumDataCSSQuery = "span.txt_query"
googleDataCSSQuery = 'div.wM6W7d span b'
youtubeDataCSSQuery = '#sbse0 > div.sbqs_c > b'


getGoogleOpenKeywordCSSQuery = 'div.s75CSd.u60jwe.r2fjmd.AB4Wff > b'
# 다음광고 선택자
DAUM_AD_normal = '.list_ad' #와이드광고, 프리미엄광고
DAUM_AD_special = '#splinkColl .list_info.mg_cont.clear' # 스페셜광고
DAUM_REL_KEYWORDS = '#netizen_lists_top span' # 관련키워드

# 검색량 선택자
# config(countGoogleSearchVolume)
volumeCSSQuery = 'div#result-stats'

# 순서 선택자
# config(checkGoogleSectionOrder)
rankCSSQuery = 'div#rso.dURPMd > div.MjjYud a'

In [29]:
# 디버그모드 설정 :
debug = True
# 데이터베이스 생성
keywords = {}
# depth 설정
maxDepth = 2
# 작성주제 생성
targets = ['스포티파이', '아디다스', '애플']
# 2차 검색 주제 설정
subjects = [
'창업자',
'브랜드 스토리',
'탄생',
# '광고',
'철학',
'논란',
# '브랜딩',
# '가치',
# '의미',
'원칙',
'역사'
]
# subjects = []
limit = 2

In [30]:
driver = webdriver.Chrome()
for target in targets :    
    if debug : print('Now Target is : ',target)
    keywords[target] = {'totalKeywords' : {}}
    subjects = [target+' '+subject for subject in subjects]      
    searchKeywords = ''
    if debug : print('TotalSubjects is : ', subjects)
    for subject in subjects :
        keywords[target][subject] = searchKeywordFormat.copy()
        if debug : print('Now Subject is : ', subject)
        for depth in range(maxDepth) :
            if debug : print('now Depth is : ', depth)
            if depth == 0 :
                if debug : print('this is first trial')                
                searchKeywords = [subject].copy()                
                if debug : print('*******all keywords we search is ', searchKeywords)
            elif depth > 0 :
                BefSearchKeywords = searchKeywords.copy()                
                searchKeywords = list(keywords[target][subject].keys())
                for w in BefSearchKeywords :
                    if w in searchKeywords :
                        searchKeywords = searchKeywords.remove(w)
                if debug : print('*******all keywords we search is ', searchKeywords)
            
            # searchKeywords = ['파타고니아', '파타고니아+창업주', '파타고니아+창업스토리', '파타고니아+핵심', '파타고니아+명언']
            # searchKeywords = [word.replace(' ', '+') for word in searchKeywords] # 뛰어쓰기 처치            
            if debug : display(pd.DataFrame(keywords[target]))
            # 키워드 분석 시작
            for searchKeyword in searchKeywords : # 주제어에 대한 반복문                
                if debug : print('NOW PROCESSING THIS (target, searchKeyword, depth): ',target,searchKeyword,depth)

                # 데이터 수집 - 뷰티풀스프
                
                daumOpenKeywords = getOpenKeywords(createDaumURL(searchKeyword), daumOpenKeywordsTag)
                daumOpenKeywords = limitLength(daumOpenKeywords, limit)
                # if debug : print('Collecting naverOpenKeywords..')
                # naverOpenKeywords = getOpenKeywords(createNaverURL(searchKeyword), naverOpenKeywordsTag)
                if debug : print('daum open keywords complete')

                # 데이터수집 - 셀레니움
                
                try:
                    # 1. 대상 웹사이트 접속
                    driver.get(createDaumURL(searchKeyword))
                    daumSuggestKeywords = seleniumClickAndCrawling(driver, daumClickElementCSSQuery, daumDataCSSQuery)
                    daumSuggestKeywords = limitLength(daumSuggestKeywords, limit)
                    if debug : print('daum suggest keywords complete')
                    driver.get(createGoogleURL(searchKeyword))
                    googleSuggestKeywords = seleniumClickAndCrawling(driver, googleClickElementCSSQuery, googleDataCSSQuery)
                    googleSuggestKeywords = limitLength(googleSuggestKeywords, limit)
                    if debug : print('google suggest keywords complete')
                    googleSuggestKeywords = [searchKeyword+" "+word for word in googleSuggestKeywords] # 구글 특수처리                    
                    if debug : print('google suggest keywords modifying complete')
                    googleOpenKeywords = getGoogleOpenKeywords(driver,searchKeyword, getGoogleOpenKeywordCSSQuery)
                    googleOpenKeywords = limitLength(googleOpenKeywords, limit)
                    if debug : print('google open keywords complete')
                    
                    # if debug : print('youtube')
                    # driver.get(createYoutubeURL(searchKeyword))
                    # youtubeSuggestKeywords = seleniumClickAndCrawling(driver, youtubeClickElementCSSQuery, youtubeDataCSSQuery)
                    
                    # 에러 핸들링
                except Exception as e:
                    if debug : print('[error occured]', e)
                    continue
                # 셀레니움 종료
                # finally:
                #     driver.quit()
                datas = [daumOpenKeywords,
                        daumSuggestKeywords,
                        googleSuggestKeywords,
                        googleOpenKeywords,
                        #  youtubeSuggestKeywords
                        ] #네이버빠짐
                
                merged_dict = {}
                for provider,data in zip(providers,datas) :            
                    submittedkeywordValuationFormat = keywordValuationFormat.copy()
                    submittedkeywordValuationFormat['provider'] = provider
                    submittedkeywordValuationFormat['depth'] = depth
                    if debug : print('modifying format complete')
                    dictionaryTypeData = keywordFormatting(data, submittedkeywordValuationFormat)
                    if debug : print('apply format complete (list -> dict)')
                    if dictionaryTypeData :
                        merged_dict.update(dictionaryTypeData)
                    if debug : print('creating merged_dict & saving complete')
                if debug : print('this is completed merged_Dict', merged_dict)
                keywords[target][subject].update(merged_dict)
                if debug : display(pd.DataFrame(keywords[target]))
           
                # 광고갯수 측정 : beautifulsoup                
                for key, value in keywords[target][subject].items() :
                    count = countDaumADs(createDaumURL(key), key, [DAUM_AD_special,DAUM_AD_normal])
                    if debug : print('ad counting complete. number of ad :', count, 'at :', key)
                    keywords[target][subject][key]['countDaumADs'] = count
                    
                
                #검색게시물 수랑 순서            
                # driver = webdriver.Chrome()
                for key, value in keywords[target][subject].items() :
                    try :
                        time.sleep(2)
                        driver.get(createGoogleURL(key)) # 디렉션
                        volume = countGoogleSearchVolume(driver,volumeCSSQuery)    # 검색량 측정
                        if debug : print('counting search volume complete volume : ', volume, 'at : ',key)
                        rank = checkGoogleSectionOrder(driver, rankCSSQuery) # 랭크 측정                
                        if debug : print('checking tistory rank complete. rank : ', rank, 'at : ',key)
                        keywords[target][subject][key]['searchVolume'] = volume # 검색량 저장
                        keywords[target][subject][key]['tistoryRank'] = rank # 랭크 저장
                        if debug : print('rank and volume saving complete')
                    except Exception as e:
                        if debug : print('[[error occured]]', e)
                
                
                
                keywords[target]['totalKeywords'].update(keywords[target][subject])
                if debug : print('merge to totalKeywords compelte')
                if debug : display(pd.DataFrame(keywords[target]))

driver.quit()


Now Target is :  스포티파이
TotalSubjects is :  ['스포티파이 창업자', '스포티파이 브랜드 스토리', '스포티파이 탄생', '스포티파이 철학', '스포티파이 논란', '스포티파이 원칙', '스포티파이 역사']
Now Subject is :  스포티파이 창업자
now Depth is :  0
this is first trial
*******all keywords we search is  ['스포티파이 창업자']


,totalKeywords,스포티파이 창업자


NOW PROCESSING THIS (target, searchKeyword, depth):  스포티파이 스포티파이 창업자 0
from https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=스포티파이 창업자
getted openKeywords length 2
daum open keywords complete
click okay
getted suggest keywords 0
daum suggest keywords complete
click okay
getted suggest keywords 10
google suggest keywords complete
google suggest keywords modifying complete
google open keywords complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
modifying format complete
apply format complete (list -> dict)
creating merged_dict & saving complete
this is completed merged_Dict {'미국 스포티파이': {'searchVolume': '', 'countDaumADs': '', 'tistoryRank': '', 'provider': 'daumOpenKeywords', 'depth': 0}, '스포티파이 창업자 창업 스토리': {'searc

,totalKeywords,스포티파이 창업자
미국 스포티파이,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
스포티파이 창업자 창업 스토리,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
스포티파이 창업자 스포티 파이 학생,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
스포티파이 창업자 스포티파이창업스토리,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
스포티파이 창업자 학생,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."


ad counting complete. number of ad : 0 at : 미국 스포티파이
ad counting complete. number of ad : 0 at : 스포티파이 창업자 창업 스토리
ad counting complete. number of ad : 0 at : 스포티파이 창업자 스포티 파이 학생
ad counting complete. number of ad : 0 at : 스포티파이 창업자 스포티파이창업스토리
ad counting complete. number of ad : 0 at : 스포티파이 창업자 학생
counting search volume complete volume :  1060000 at :  미국 스포티파이
checking tistory rank complete. rank :  1 at :  미국 스포티파이
rank and volume saving complete
counting search volume complete volume :  13400 at :  스포티파이 창업자 창업 스토리
checking tistory rank complete. rank :  3 at :  스포티파이 창업자 창업 스토리
rank and volume saving complete
counting search volume complete volume :  13800 at :  스포티파이 창업자 스포티 파이 학생
checking tistory rank complete. rank :  19 at :  스포티파이 창업자 스포티 파이 학생
rank and volume saving complete
counting search volume complete volume :  13800 at :  스포티파이 창업자 스포티파이창업스토리
checking tistory rank complete. rank :  4 at :  스포티파이 창업자 스포티파이창업스토리
rank and volume saving complete
counting search volume comp

...

,totalKeywords,스포티파이 창업자
미국 스포티파이,"{'searchVolume': 1060000, 'countDaumADs': 0, '...","{'searchVolume': 1060000, 'countDaumADs': 0, '..."
스포티파이 창업자 창업 스토리,"{'searchVolume': 13400, 'countDaumADs': 0, 'ti...","{'searchVolume': 13400, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 스포티 파이 학생,"{'searchVolume': 13800, 'countDaumADs': 0, 'ti...","{'searchVolume': 13800, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 스포티파이창업스토리,"{'searchVolume': 13800, 'countDaumADs': 0, 'ti...","{'searchVolume': 13800, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 학생,"{'searchVolume': 15000, 'countDaumADs': 0, 'ti...","{'searchVolume': 15000, 'countDaumADs': 0, 'ti..."
스포티파이 순위,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
스포티파이 가격,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
미국 스포티파이 순위,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
미국 스포티파이 무료,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."
미국 스포티파이 스포티파이순위,NaN,"{'searchVolume': '', 'countDaumADs': '', 'tist..."


ad counting complete. number of ad : 0 at : 미국 스포티파이
ad counting complete. number of ad : 0 at : 스포티파이 창업자 창업 스토리
ad counting complete. number of ad : 0 at : 스포티파이 창업자 스포티 파이 학생
ad counting complete. number of ad : 0 at : 스포티파이 창업자 스포티파이창업스토리
ad counting complete. number of ad : 0 at : 스포티파이 창업자 학생
ad counting complete. number of ad : 1 at : 스포티파이 순위
ad counting complete. number of ad : 0 at : 스포티파이 가격
ad counting complete. number of ad : 0 at : 미국 스포티파이 순위
ad counting complete. number of ad : 0 at : 미국 스포티파이 무료
ad counting complete. number of ad : 0 at : 미국 스포티파이 스포티파이순위
ad counting complete. number of ad : 0 at : 미국 스포티파이 스포티파이무료
counting search volume complete volume :  1060000 at :  미국 스포티파이
checking tistory rank complete. rank :  1 at :  미국 스포티파이
rank and volume saving complete
counting search volume complete volume :  13400 at :  스포티파이 창업자 창업 스토리
checking tistory rank complete. rank :  3 at :  스포티파이 창업자 창업 스토리
rank and volume saving complete
counting search volume complete volume

,totalKeywords,스포티파이 창업자
미국 스포티파이,"{'searchVolume': 1060000, 'countDaumADs': 0, '...","{'searchVolume': 1060000, 'countDaumADs': 0, '..."
스포티파이 창업자 창업 스토리,"{'searchVolume': 13400, 'countDaumADs': 0, 'ti...","{'searchVolume': 13400, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 스포티 파이 학생,"{'searchVolume': 13800, 'countDaumADs': 0, 'ti...","{'searchVolume': 13800, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 스포티파이창업스토리,"{'searchVolume': 13800, 'countDaumADs': 0, 'ti...","{'searchVolume': 13800, 'countDaumADs': 0, 'ti..."
스포티파이 창업자 학생,"{'searchVolume': 15000, 'countDaumADs': 0, 'ti...","{'searchVolume': 15000, 'countDaumADs': 0, 'ti..."
스포티파이 순위,"{'searchVolume': 831000, 'countDaumADs': 1, 't...","{'searchVolume': 831000, 'countDaumADs': 1, 't..."
스포티파이 가격,"{'searchVolume': 652000, 'countDaumADs': 0, 't...","{'searchVolume': 652000, 'countDaumADs': 0, 't..."
미국 스포티파이 순위,"{'searchVolume': 945000, 'countDaumADs': 0, 't...","{'searchVolume': 945000, 'countDaumADs': 0, 't..."
미국 스포티파이 무료,"{'searchVolume': 895000, 'countDaumADs': 0, 't...","{'searchVolume': 895000, 'countDaumADs': 0, 't..."
미국 스포티파이 스포티파이순위,"{'searchVolume': 1230000, 'countDaumADs': 0, '...","{'searchVolume': 1230000, 'countDaumADs': 0, '..."


NOW PROCESSING THIS (target, searchKeyword, depth):  스포티파이 스포티파이 창업자 창업 스토리 1
from https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=스포티파이 창업자 창업 스토리
getted openKeywords length 0


TypeError: object of type 'NoneType' has no len()

In [35]:
a = None
type(a)

NoneType

In [37]:

if a :
    if debug : print('true')
else :
    if debug : print('false')

b


In [ ]:
for key, val in keywords :
    val['totalKeywords'].to_csv(f'{key}.csv')

NameError: name 'keywords' is not defined

In [1]:
1+1

2

In [2]:
fron = ['a']

In [5]:

'b' not in fron

True